# The dockstring _de novo_ molecular design (aka molecular optimization) benchmark

### Summary

In this tutorial we evaluate a dummy algorithm on a _mini version_ of dockstring's de novo molecular design benchmarks.
These methods are very simple, and the performance is expected to be very poor.
The actual dockstring benchmark has a budget of 5000 calls to the objective function,
which takes a very long time to run (at least 1 day).
For the purposes of this tutorial, we limit the budget to just 5 objective function evaluations,
but if you are using this notebook to test your method then you should replace this.

### Method tested

The method is a model-free method which uses the dataset of pre-computed values, and treats the objective functions as _white box_ (i.e. one is allowed to directly use the docking and QED scores used to compute the objective function value). It randomly chooses one of the top 10,000 known SMILES, modifies it by adding a single carbon atom to the beginning of the string, and returns it if the resulting SMILES is valid _and_ it if its QED value does not change too much. It can be thought of as a very simple genetic algorithm.

### FAQ

If you are replacing this code with your own algorithm, here are the answers to some potential questions that you may have:

- _Do I need to use the dataset_? No, your method can ignore the dataset if if it not needed.
- _Is there a limit on how often the QED function can be called?_: No, this function is very fast. The limit only applies to the docking functions
- _Do I need to use the benchmark API provided by dockstring?_ It is highly recommended but not strictly necessary.
- _Do I need to use LRU cache?_ No, but this option is "safer" than counting function calls manually because it makes it very difficult to count incorrectly.
- _For objectives involving multiple calls to dockstring (e.g. selective JAK2 which docks JAK2 and LCK), can we only call dockstring on a subset of the objectives?_ For comparing to other methods on the original benchmark, our suggestion is _no_, but feel free to experiment with this problem setting yourself!

In [ ]:
OBJECTIVE_EVAL_BUDGET = 5
# OBJECTIVE_EVAL_BUDGET = 5000  # *uncomment this line to run the true benchmark*

In [ ]:
# Import everything- install missing libraries as needed
# (everything here is quite standard)
import random
import functools

from pathlib import Path
import numpy as np
import pandas as pd
from rdkit import Chem
from tqdm.auto import tqdm  # Helpful progress bars

## 1. Import dockstring benchmark functions

The dockstring package has an API for accessing the benchmark objective functions.
We demonstrate its use below.

In [ ]:
from dockstring.benchmarks.original import get_benchmark_functions

In [ ]:
# Load the benchmark functions, optionally specifying the number of CPUs
# Result is a dictionary containing 3 callable objects
benchmark_function_dict = get_benchmark_functions(num_cpus=8)  # TODO: adjust if needed
print(f"Keys: {benchmark_function_dict.keys()}")
benchmark_function_dict

In [ ]:
# The main method of interacting with these functions is calling them with a SMILES string
# They return a tuple (v, d) where v is the objective value to be minimized,
# and d is a dictionary of intermediate values
sample_benchmark_fn = benchmark_function_dict["selective_JAK2"]
sample_smiles = "CCCC"
sample_benchmark_fn(sample_smiles)

In [ ]:
# If desired, the objective can be computed from the intermeidate values as so:
sample_benchmark_fn.aggregation_function(QED=0.4310, JAK2=-2.7, LCK=-2.6)

In [ ]:
# The list of required intermediate values can be viewed as so:
sample_benchmark_fn.base_functions.keys()

In [ ]:
# If desired, the functions can be called individually as so:
sample_benchmark_fn.base_functions["JAK2"](sample_smiles)

In [ ]:
# Cleanup, so the rest of the notebook is not affected
del sample_benchmark_fn, sample_smiles

## 2. Downloading the dockstring dataset

This is only necessary if your algorithm will use the pre-computed values in the dockstring dataset.
If you haven't downloaded the dataset,
follow the instructions [here](https://github.com/dockstring/dataset)
to download the data into a folder called `data` in the root of this repository.
If this is done correctly, the following cell should run without error.

In [ ]:
# If the necessary data files are present, this cell will run without error
dataset_path = Path("../data/dockstring-dataset.tsv")
assert dataset_path.exists()

## 3. Loading the dataset and filling in known objective function values

We read in the `.tsv` file, fill in the QED values for all molecules, and compute the objective values using the API above.

In [ ]:
# Read in the dataset using pandas (using "\t" since it is tab-delimited)
df = pd.read_csv(dataset_path, sep="\t").set_index("inchikey")
df

In [ ]:
# Calculate the QED for the dataset
df["QED"] = [Chem.QED.qed(Chem.MolFromSmiles(str(s))) for s in tqdm(df.smiles)]
df

In [ ]:
# Insert values for all objective functions into the dataframe
for benchmark_name, benchmark_func in benchmark_function_dict.items():
    print(benchmark_name)
    
    # Get all the required values from the dataframe
    required_values = {prop_name: df[prop_name].values for prop_name in benchmark_func.base_functions.keys()}
    
    # Calculate objective function values
    objective_values = [benchmark_func.aggregation_function(**{k: v[i] for k, v in required_values.items()}) for i in range(len(df))]
    
    # Append these to the dataframe, using the prefix "obj" to avoid name collisions
    df[f"obj_{benchmark_name}"] = objective_values
    
    del benchmark_name, benchmark_func, objective_values, required_values
df

In [ ]:
# Confirm that the pre-computed values match the real objective functions for an arbitrary SMILES in the dataset
arbitrary_row = df.iloc[42]
for benchmark_name, benchmark_func in benchmark_function_dict.items():
    objective_val, _ = benchmark_func(arbitrary_row.smiles)
    assert np.isclose(objective_val, arbitrary_row[f"obj_{benchmark_name}"], atol=1e-2, rtol=1e-2)

## 5. Run the methods and print results (the top molecules)

We use a LRU cache to ensure that we correctly count the number of calls to the objective functions,
and track the top molecules over time to provide insight into how performance changes over time.
Note that the code for the actual algorithm and the result aggregation is in another file.

In [ ]:
# Some functions for printing results
EVAL_TIMES = [3, 4, OBJECTIVE_EVAL_BUDGET]
TOP_K = 3
from de_novo_design_utils import generate_smiles, topk_mols_over_time, print_results_over_time

In [ ]:
for benchmark_name, benchmark_func in benchmark_function_dict.items():
    print(f"## Start benchmark {benchmark_name} ##")
    random.seed(0)
    np.random.seed(0)
    
    # Initialize the dataset
    dataset = {
        row["smiles"]: (
            row[f"obj_{benchmark_name}"], 
            {fname: row[fname] for fname in benchmark_func.base_functions.keys()}
        )
        for _, row in df.iterrows()
    }
    
    # Use LRU cache to continue until the evaluation budget is reached.
    # LRU cache prevents us from accidentally doing additional function calls.
    cached_benchmark_func = functools.lru_cache(maxsize=None)(benchmark_func)
    smiles_to_results = dict()  # This is where we store the results
    while cached_benchmark_func.cache_info().misses < OBJECTIVE_EVAL_BUDGET:
        
        # Step 1: choose a SMILES to evaluate
        # TODO: Replace this part with your algorithm
        chosen_smiles = generate_smiles(dataset)
        
        # Step 2: evaluate objective function, if it has not already been evaluated and is not in the dataset
        if chosen_smiles not in smiles_to_results and chosen_smiles not in dataset:
        
            # Call the benchmark function
            current_result = cached_benchmark_func(chosen_smiles)

            # Store results of function, and also how many times
            # the function was called when this result was obtained
            smiles_to_results[chosen_smiles] = (
                current_result, cached_benchmark_func.cache_info().misses
            )
            
            # Update the dataset with the new result
            # TODO: replace this with your own dataset update
            dataset[chosen_smiles] = current_result
            
            del current_result
        
        # Cleanup after this iteration
        del chosen_smiles
    
    # Print results for this benchmark
    results_over_time = topk_mols_over_time(smiles_to_results, times=EVAL_TIMES, k=TOP_K)
    print_results_over_time(results_over_time)